In [72]:
import os
from tqdm import tqdm
from PIL import Image
import shutil
import random

DESTINATION_DIR = 'YOLO'

def convert_file(dataset_dir: str, file: str, obj_class: str, train_split: float):
    full_path = os.path.join(dataset_dir, file)
    fname, img_dir, destination = [f"{full_path}/{file}.txt", f"{full_path}/img", f"{full_path}/bbox"]
    if not os.path.exists(destination):
        os.makedirs(destination)
    
    images = [d for d in os.listdir(img_dir)]

    with open(fname, 'r') as f:
        num_lines = sum([1 for l in f if l.strip() != ''])

    with open(fname, 'r') as f:
        if num_lines != len(images):
            print(f"length of {fname} does not match number of files in {img_dir}")
        else:
            lines = [l for l in f]
            for line, image in zip(lines, images):
                if line.strip() == '': pass

                x, y, w, h = [float(val) for val in line.strip().replace(' ', '\t').replace(',', '\t').split('\t')]
                x, y, w, h = normalise_bbox(x, y, w, h, os.path.join(img_dir, image))
                new_fname = f"{file}_{image.replace('.jpg', '')}"

                split = 'train' if random.random() < train_split else 'val'

                with open(os.path.join(dataset_dir, '..', DESTINATION_DIR, 'labels', split, (new_fname+'.txt')), 'w') as f2:
                    f2.write(' '.join(str(v) for v in [obj_class, x, y, w, h]))
                shutil.copyfile(os.path.join(img_dir, image), os.path.join(dataset_dir, '..', DESTINATION_DIR, 'images', split, (new_fname+'.jpg')))

def normalise_bbox(x, y, w, h, fname):
    with Image.open(fname) as img:
        width, height = img.size
    return (x / width, y / height, w / width, h / height)

def convert_dataset(dataset_dir: str = '.', train_split: float = 0.8):
    
    create_directory_if_not_exists(os.path.join(dataset_dir, DESTINATION_DIR, 'images', 'train'))
    create_directory_if_not_exists(os.path.join(dataset_dir, DESTINATION_DIR, 'images', 'val'))
    create_directory_if_not_exists(os.path.join(dataset_dir, DESTINATION_DIR, 'labels', 'train'))
    create_directory_if_not_exists(os.path.join(dataset_dir, DESTINATION_DIR, 'labels', 'val'))
    dataset_dir = os.path.join(dataset_dir, 'raw')
    directories = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    for d in tqdm(directories):
        convert_file(dataset_dir, d, "0" if "aircraft" in d else "1", train_split=train_split)


def create_directory_if_not_exists(directory_path):
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)


In [73]:
convert_dataset('datasets/ToT_extracted')

100%|██████████| 37/37 [00:03<00:00, 10.27it/s]


In [65]:
os.listdir(os.path.join('datasets', '..'))
# ValueError: could not convert string to float: '477.17\t205.83\t9.7367\t10.023'

['VideoTransform.py',
 'VideoMix.py',
 '.venv',
 '.git',
 'train.py',
 'yolov8n.pt',
 'datasets',
 'runs',
 'README.md',
 'i3d.py',
 'YOLO_training.ipynb',
 'convert_baseline.ipynb']

In [41]:
with open('datasets/ToT_extracted/bird22-1/bird22-1.txt', 'r') as f1:
    print(sum(1 for _ in f1))
with open('datasets/ToT_extracted/bird22-1/bird22-1.txt', 'r') as f1:
    print(sum([1 for _ in f1]))

119
119


convert_baseline.ipynb	README.md  VideoMix.py		yolov8n.pt
datasets		runs	   VideoTransform.py
i3d.py			train.py   YOLO_training.ipynb
